In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=1000)
tf.enable_eager_execution()
print(tf.__version__)

In [ ]:
# read_data_sets('download_directory')
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

In [ ]:
# next_batch(1) : 학습 데이터에서 1개의 X(독립변수), y(종속변수) 반환
X, y = mnist.train.next_batch(1)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
# image를 출력하기 위해 28x28의 array로 변환
X.reshape((28, 28))

In [ ]:
plt.imshow(X.reshape((28, 28)), cmap='Greys')
plt.show()

In [ ]:
X = X.reshape(1, 28, 28, 1)
X.shape

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

In [ ]:
# [fil_idx, fil_col, col_type, fil_cnt] 
# 	-> fil_idx : 필터 세로, fil_col : 필터 가로, col_tye : 1(흑백), 3(컬러), fil_cnt : 필터 갯수
w1 = tf.Variable(initializer([3, 3, 1, 32]))
w1

In [ ]:
# 0번째 필터 조회
w1[ : , : , : , 0]

In [ ]:
tf.reshape(w1[ : , : , : , 0], (3, 3))

In [ ]:
for i in range(32) :
	print(f'filter i = {i}')
	print('=' * 100)
	print(tf.reshape(w1[ : , : , : , i], (3, 3)))
	print('=' * 100)

In [ ]:
img = X

In [ ]:
img

In [ ]:
plt.imshow(img.reshape((28, 28)), cmap='Greys')
plt.show()

In [ ]:
# convolution을 하기 위해 4차원 array로 변환
img = img.reshape(1, 28, 28, 1)

In [ ]:
# convolution 실행
convolution_h = tf.nn.conv2d(X, w1, strides=1, padding='SAME')
convolution_h

In [ ]:
# convolution 실행 후 결과 출력
convolution_h[0, : , : , 0]

In [ ]:
# convolution 실행 후 0번째 결과 시각화
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 2], cmap='Greys')
plt.show()

In [ ]:
# convolution 실행 후 결과들 시각화
fig = plt.figure(figsize=(30, 20))

for i in range(32) :
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, : , : , i], cmap='Greys')

In [ ]:
# activation function relu 실행
# 음수 -> 0
# 0, 양수 -> 그대로
convolution_h = tf.nn.relu(convolution_h)
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
# relu 실행 후 0번째 출력
convolution_h[0, : , : , 0]

In [ ]:
# relu 실행 후 0번째 결과 시각화
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
# relu 실행 후 결과들 시각화
fig = plt.figure(figsize=(30, 20))

for i in range(32) :
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, : , : , i], cmap='Greys')

In [ ]:
# max_pool 실행
convolution_h = tf.nn.max_pool(convolution_h, ksize=2, strides=1 , padding='SAME')
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
# max_pooling 결과 출력
convolution_h[0, : , : , 0]

In [ ]:
# max_pooling 실행 후 0번째 결과 시각화
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 2], cmap='Greys')
plt.show()

In [ ]:
# max_pooling 결과들 시각화
fig = plt.figure(figsize=(30, 20))

for i in range(32) : 
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, : , : , i], cmap='Greys')

In [ ]:
convolution_h = tf.reshape(convolution_h, [-1, 28 * 28 * 32])
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
w3 = tf.Variable(initializer([28 * 28 * 32, 10]))
w3

In [ ]:
w3.shape

In [ ]:
y_h1 = tf.matmul(convolution_h, w3)

In [ ]:
y_h1.shape

In [ ]:
y_h1 = tf.nn.softmax(y_h1)
y_h1

In [ ]:
# mnist training data
mnist.train.num_examples

In [ ]:
# 학습시킬 image 크기를 묶음
batch_size = 100

total_batch = int(mnist.train.num_examples / batch_size)
print('total_batch = ', total_batch)

In [ ]:
# learning_rate 설정
optimizer = tf.train.AdamOptimizer(0.001)

In [ ]:
for step in range(5) :

	total_cost = 0

	for i in range(total_batch) : 
		X, y = mnist.train.next_batch(batch_size)
		X = X.reshape(-1, 28, 28, 1)

		with tf.GradientTape() as tape :
			convolution_h = tf.nn.conv2d(X, w1, strides=1, padding='SAME')
			convolution_h = tf.nn.relu(convolution_h)
			convolution_h = tf.nn.max_pool(convolution_h, ksize=2, strides=1, padding='SAME')
			convolution_h = tf.reshape(convolution_h, [-1, 28 * 28 * 32])
			
			y_h1 = tf.matmul(convolution_h, w3)
			y_h1 = tf.nn.softmax(y_h1)

			cost = -tf.reduce_sum(y * tf.log(y_h1) / len(y_h1))

			grads = tape.gradient(cost, [w1, w3])

		optimizer.apply_gradients(grads_and_vars=zip(grads, [w1, w3]))
		total_cost += cost
	
	print(f'step : {step}, cost : {total_cost / total_batch}')


In [ ]:
img

In [ ]:
plt.imshow(img.reshape((28, 28)), cmap='Greys')
plt.show()

In [ ]:
w1

In [ ]:
w1[ : , : , : , 0]

In [ ]:
tf.reshape(w1[ : , : , : , 0], (3, 3))

In [ ]:
for i in range(32) :
	print('filter = ', i)
	print('=' * 100)
	print(tf.reshape(w1[ : , : , : , i], (3, 3)))
	print('=' * 100)

In [ ]:
img = img.reshape(1, 28, 28, 1)

In [ ]:
convolution_h = tf.nn.conv2d(img, w1, strides=1, padding='SAME')
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
convolution_h[0, : , : , 0]

In [ ]:
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 2], cmap='Greys')
plt.show()

In [ ]:
fig = plt.figure(figsize=(30, 20))

for i in range(32) :
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, : , : , i], cmap='Greys')

In [ ]:
convolution_h = tf.nn.relu(convolution_h)
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
convolution_h[0, : , : , 0]

In [ ]:
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 2], cmap='Greys')
plt.show()

In [ ]:
fig = plt.figure(figsize=(30, 20))

for i in range(32) :
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, :, :, i], cmap='Greys')

In [ ]:
convolution_h = tf.nn.max_pool(convolution_h, ksize=2, strides=1, padding='SAME')
convolution_h

In [ ]:
convolution_h.shape

In [ ]:
convolution_h[0, : , : , 0]

In [ ]:
plt.imshow(convolution_h[0, : , : , 0], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 1], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(convolution_h[0, : , : , 2], cmap='Greys')
plt.show()

In [ ]:
fig = plt.figure(figsize=(30, 20))

for i in range(32) :
	subplot = fig.add_subplot(4, 8, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.imshow(convolution_h[0, : , : , i], cmap='Greys')

In [ ]:
w3

In [ ]:
test_images = mnist.test.images[0 : 10000]
test_images

In [ ]:
test_labels = mnist.test.labels[0 : 10000]
test_labels

In [ ]:
test_images = test_images.reshape(-1, 28, 28, 1)
test_images.shape

In [ ]:
convolution_h = tf.nn.conv2d(test_images, w1, strides=1, padding='SAME')
convolution_h

In [ ]:
convolution_h = tf.nn.relu(convolution_h)

In [ ]:
convolution_h = tf.nn.max_pool(convolution_h, ksize=2, strides=1, padding='SAME')
convolution_h = tf.reshape(convolution_h, [-1, 28 * 28 * 32])

In [ ]:
y_h1 = tf.matmul(convolution_h, w3)
predict = tf.nn.softmax(y_h1)
predict

In [ ]:
predict01 = tf.argmax(predict, 1)
predict01

In [ ]:
target = tf.argmax(test_labels, 1)
target

In [ ]:
ac01 = tf.equal(predict01, target)
ac01

In [ ]:
ac02 = tf.cast(ac01, dtype='float32')
ac02

In [ ]:
ac03 = tf.reduce_mean(ac02)
ac03

In [ ]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict01, target), dtype='float32'))
print('=' * 100)
print('accuracy')
print(accuracy.numpy())

In [ ]:
fig = plt.figure(figsize=(30, 20))

for i in range(30) :
	subplot = fig.add_subplot(6, 5, i + 1)
	subplot.set_xticks([])
	subplot.set_yticks([])
	subplot.set_title('predict : %d' % predict01[i])
	subplot.imshow(mnist.test.images[i].reshape((28, 28)), cmap='Greys')
plt.show()